# Project Description

Análise do risco de inadimplência dos mutuários

Seu projeto é preparar um relatório para a divisão de empréstimos de um banco. Você precisará descobrir se o estado civil de um cliente e o número de filhos têm impacto sobre se ele deixará de pagar um empréstimo. O banco já tem alguns dados sobre a capacidade de crédito dos clientes.

Seu relatório será considerado ao construir a pontuação de crédito de um cliente em potencial. A pontuação de crédito é usada para avaliar a capacidade de um devedor em potencial de pagar seu empréstimo.

Neste caderno, você recebe dicas, instruções breves e sugestões de raciocínio. Não os ignore, pois eles são projetados para te equipar com a estrutura do projeto e o ajudarão a analisar o que você está fazendo em um nível mais profundo. Antes de enviar seu projeto, certifique-se de remover todas as dicas e descrições fornecidas a você. Em vez disso, faça com que este relatório pareça que você está enviando para seus colegas de equipe para demonstrar suas descobertas - eles não devem saber que você teve qualquer ajuda externa nossa! Para ajudá-lo, colocamos as dicas que você deve remover entre colchetes.]

Antes de mergulhar na análise dos seus dados, explique os propósitos do projeto e as hipóteses que você testará.

# Initializing

In [4]:
# importing library / importando biblioteca

import pandas as pd


In [5]:
# reading file and saving as 'cs' (credit_scoring) / lendo arquivo e salvando como 'cs'

try:
    cs = pd.read_csv("/datasets/credit_scoring_eng.csv")

except FileNotFoundError:
    
    cs = pd.read_csv("credit_scoring_eng.csv")

In [6]:
# requesting file information / requisitando descrição de dados

cs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Descrição dos dados**

- `children` - number of children per family / o número de crianças na família 
- `days_employed` - number of days employed / experiência de trabalho em dias
- `dob_years` - employee age / idade do cliente em anos
- `education` - education level / educação do cliente
- `education_id` - education id / identificador de educação
- `family_status` - family status / estado civil do cliente
- `family_status_id` - family status id / identificador de estado civil
- `gender` - gender / gênero do cliente
- `income_type` -income type / tipo de emprego
- `debt` - debt to pay loan, being 1 for 'yes' and 0 for 'no debt' / havia alguma dívida no pagamento do empréstimo, sendo 1 para 'sim' e 0 para 'sem divida'
- `total_income` - year income / renda anual
- `purpose` - reason to get a loan / o objetivo de obter um empréstimo

In [8]:
# reading 20 first lines / lendo as primeiras 20 linhas

cs.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


## Analise primaria / First Analysis 


Olhando coluna por coluna é possivel notar:
- days_employed, com valores negativos e valores absurdamente altos.
- dob_years, nome da coluna pode ser melhorado para 'age'
- education, contém strings maiúsculas e minúsculas
- purpose, contém diversas formas de descrever a razões similares pelas quais os emprestímos foram feitos, e preciso simplificar


Looking column by column is possible to note:
- days_employed has negative values and extremely high values
- dob_years, name of column can be substituted by 'age'
- education, has upper case and lower case strings
- purpose, has many ways to describe similar reasons to take a loan, needs simplification





In [10]:
# requesting description / requerindo descricao

cs.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


In [11]:
# renaming column 'dob_years' to 'age' / renomeando coluna 'dob_years para 'age'

cs = cs.rename(columns={'dob_years' : 'age' })

cs.columns

Index(['children', 'days_employed', 'age', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [12]:
# identifying possible wrong values in 'children' collumn / identificando possiveis valores errados na coluna 'children'

cs['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [13]:
# converting negative numbers in 'children' column to positive / convertendo numeros negativos em 'children' para positivo

cs['children'] = cs['children'].abs()
    

In [14]:
cs['children'] = cs['children'].replace(20, 2)

cs['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [15]:
# formating strings to lower case in 'education'  / formatando strings para letras minusculas na coluna education

cs['education'] = cs['education'].str.lower()

cs['education'].value_counts()


secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

In [16]:
# identifying null values in all columns / identificando valores nulos nas colunas

cs.isnull().sum()


children               0
days_employed       2174
age                    0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

- it looks like for every null value in 'days_employed' we have a null value in 'total_income' 
- parece que para cada valor nulo em 'days_employed existe um valor nulo em 'total_income

In [18]:
# finding duplicated values in 'cs' / achando valores duplicados em 'cs'

duplicated_cs = cs[cs.duplicated()]

duplicated_cs.head(10)



,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,NaN,41,secondary education,1,married,0,F,employee,0,NaN,purchase of the house for my family
3290,0,NaN,58,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding
4182,1,NaN,34,bachelor's degree,0,civil partnership,1,F,employee,0,NaN,wedding ceremony
4851,0,NaN,60,secondary education,1,civil partnership,1,F,retiree,0,NaN,wedding ceremony
5557,0,NaN,58,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding
6312,0,NaN,30,secondary education,1,married,0,M,employee,0,NaN,building a real estate
7808,0,NaN,57,secondary education,1,civil partnership,1,F,retiree,0,NaN,having a wedding
7921,0,NaN,64,bachelor's degree,0,civil partnership,1,F,retiree,0,NaN,having a wedding
7938,0,NaN,71,secondary education,1,civil partnership,1,F,retiree,0,NaN,having a wedding
8583,0,NaN,58,bachelor's degree,0,unmarried,4,F,retiree,0,NaN,supplementary education


In [19]:
# identifying similar duplicates / identificando duplicados similares

cs['purpose'].sort_values().unique()

array(['building a property', 'building a real estate',
       'buy commercial real estate', 'buy real estate',
       'buy residential real estate', 'buying a second-hand car',
       'buying my own car', 'buying property for renting out', 'car',
       'car purchase', 'cars', 'construction of own property',
       'education', 'getting an education', 'getting higher education',
       'going to university', 'having a wedding', 'housing',
       'housing renovation', 'housing transactions', 'profile education',
       'property', 'purchase of a car', 'purchase of my own house',
       'purchase of the house', 'purchase of the house for my family',
       'real estate transactions', 'second-hand car purchase',
       'supplementary education', 'to become educated', 'to buy a car',
       'to get a supplementary education', 'to have a wedding',
       'to own a car', 'transactions with commercial real estate',
       'transactions with my real estate', 'university education',
       'we

In [20]:
# function to replace values in collumn purpose regarding property buy, to 'property'
# funcao para substituir valores na coluna 'purpose' de acordo com compra de propriedade, para 'property'

def replace_buy_purpose(wrong_purpose, correct_purpose): 
    for wrong_purpose in wrong_purpose: 
        cs['purpose'] = cs['purpose'].replace(wrong_purpose, correct_purpose)

duplicates = ['buy commercial real estate', 'buy real estate','buy residential real estate',
              'buying property for renting out', 'property', 'purchase of my own house', 'purchase of the house','purchase of the house for my family',
              'real estate transactions', 'transactions with commercial real estate', 'transactions with my real estate', 'housing transactions'            
             ] 
name = 'property' 
replace_buy_purpose(duplicates, name) 


In [21]:
# checking next similar duplicates

cs['purpose'].sort_values().unique()


array(['building a property', 'building a real estate',
       'buying a second-hand car', 'buying my own car', 'car',
       'car purchase', 'cars', 'construction of own property',
       'education', 'getting an education', 'getting higher education',
       'going to university', 'having a wedding', 'housing',
       'housing renovation', 'profile education', 'property',
       'purchase of a car', 'second-hand car purchase',
       'supplementary education', 'to become educated', 'to buy a car',
       'to get a supplementary education', 'to have a wedding',
       'to own a car', 'university education', 'wedding ceremony'],
      dtype=object)

In [22]:
# function to replace values in collumn purpose regarding building and renovation, to 'building'
# funcao para substituir valores na coluna 'purpose' de acordo com construcao ou obra, para 'building'

def replace_build_purpose(wrong_purpose, correct_purpose): 
    for wrong_purpose in wrong_purpose: 
        cs['purpose'] = cs['purpose'].replace(wrong_purpose, correct_purpose)
        
duplicates = ['building a property', 'building a real estate', 'construction of own property',
              'housing renovation', 'housing'             
             ]
name = 'building' 
replace_build_purpose(duplicates, name) 


In [23]:
# checking next similar duplicates

cs['purpose'].sort_values().unique()

array(['building', 'buying a second-hand car', 'buying my own car', 'car',
       'car purchase', 'cars', 'education', 'getting an education',
       'getting higher education', 'going to university',
       'having a wedding', 'profile education', 'property',
       'purchase of a car', 'second-hand car purchase',
       'supplementary education', 'to become educated', 'to buy a car',
       'to get a supplementary education', 'to have a wedding',
       'to own a car', 'university education', 'wedding ceremony'],
      dtype=object)

In [24]:
# function to replace values in collumn purpose regarding car buy, to 'car'
# funcao para substituir valores na coluna 'purpose' de acordo com a compra de carro, para 'car'

def replace_car_purpose(wrong_purpose, correct_purpose): 
    for wrong_purpose in wrong_purpose: 
        cs['purpose'] = cs['purpose'].replace(wrong_purpose, correct_purpose)
        
duplicates = ['buying a second-hand car', 'buying my own car', 'car', 'car purchase', 'cars', 'purchase of a car', 'second-hand car purchase',
              'to buy a car', 'to own a car'             
             ]
name = 'car' 
replace_car_purpose(duplicates, name) 




In [25]:
# checking next similar duplicates

cs['purpose'].sort_values().unique()

array(['building', 'car', 'education', 'getting an education',
       'getting higher education', 'going to university',
       'having a wedding', 'profile education', 'property',
       'supplementary education', 'to become educated',
       'to get a supplementary education', 'to have a wedding',
       'university education', 'wedding ceremony'], dtype=object)

In [26]:
# function to replace values in collumn purpose regarding education
# funcao para substituir valores na coluna 'purpose' de acordo com educacao, para 'education'

def replace_education_purpose(wrong_purpose, correct_purpose): 
    for wrong_purpose in wrong_purpose: 
        cs['purpose'] = cs['purpose'].replace(wrong_purpose, correct_purpose)
        
duplicates = ['education', 'getting an education', 'getting higher education', 'going to university', 'profile education',
              'supplementary education', 'to become educated', 'to get a supplementary education', 'university education'
             ]
name = 'education' 

replace_education_purpose(duplicates, name) 


In [27]:
# checking next similar duplicates

cs['purpose'].sort_values().unique()

array(['building', 'car', 'education', 'having a wedding', 'property',
       'to have a wedding', 'wedding ceremony'], dtype=object)

In [28]:
# function to replace values in collumn purpose regarding wedding
# funcao para substituir valores na coluna 'purpose' de acordo com casar, para 'wedding'

def replace_wedding_purpose(wrong_purpose, correct_purpose): 
    for wrong_purpose in wrong_purpose: 
        cs['purpose'] = cs['purpose'].replace(wrong_purpose, correct_purpose)
        
duplicates = ['having a wedding','to have a wedding','wedding ceremony'
             ]
name = 'wedding' 
replace_wedding_purpose(duplicates, name) 


In [29]:
# checking if all similar duplicates are sorted

cs['purpose'].unique()

array(['property', 'car', 'education', 'wedding', 'building'],
      dtype=object)

- finding what is the best value to replace 'NaN' values in 'total_income' 
- descobrindo qual melhor valor para substituir valores 'NaN' na 'total_income 

In [31]:
# identifying max, mean and median for 'total_income' 
# identificando valor maximo, media e mediana para 'total_income'

cs['total_income'].describe()


count     19351.000000
mean      26787.568355
std       16475.450632
min        3306.762000
25%       16488.504500
50%       23202.870000
75%       32549.611000
max      362496.645000
Name: total_income, dtype: float64

- 'total_income' shows a symmetrical distribution
- 'total_income' mostra distribuicao simetrica

- finding what is the best value to replace 'NaN' values in 'days_employed' 
- descobrindo qual melhor valor para substituir valores 'NaN' na 'days_employed' 

In [34]:
# identifying max, mean and median for 'days_employed'
# identificando valor maximo, media e mediana para 'days_employed'

cs['days_employed'].describe()

# 'days_employed' shows an asymmetrical distribution
# 'days_employed' mostra distribuicao assimetrica

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

In [35]:
# converting all values in 'days_employed' column to positive
# convertendo valores negativos em 'days_employed' para positivo

cs['days_employed'] = cs['days_employed'].abs()



- some values in 'days_employed' column exceed human life span, 
- I will replace them for a maximum of 18250 days worked which is '50 years'
- alguns valores em 'days_employed' excedem a expectativa de vida de um ser humano,
- vou substituir esse valores pelo maximo de 18250 dias que equivalem a 50 anos.

In [37]:
# function to convert values that exceed 14600 days, 50 years
# função para substituir valores que excedem os 14600 dias, 50 anos

def replace_days_employed(column):
    
    cs.loc[cs[column] > 14600, column] = 14600  
    
    return cs

cs = replace_days_employed('days_employed')


In [38]:
cs['days_employed'].describe()

count    19351.000000
mean      4532.289223
std       5127.487807
min         24.141633
25%        927.009265
50%       2194.220567
75%       5537.882441
max      14600.000000
Name: days_employed, dtype: float64

In [39]:
# filtrando dados em 'days_employed'  e 'age' / filtering data to 'days_employed', ' age'

age_days_filtered = cs.loc[cs['days_employed'] == 14600, ['days_employed', 'age']]

age_days_filtered.head(10)

,days_employed,age
4,14600.0,53
18,14600.0,53
24,14600.0,57
25,14600.0,67
30,14600.0,62
35,14600.0,68
50,14600.0,63
56,14600.0,64
71,14600.0,62
78,14600.0,61


In [40]:
# requisitando descricao da variavel filtrada / requesting description of filtered variable

age_days_filtered.describe()

,days_employed,age
count,3462.0,3462.000000
mean,14600.0,59.131427
std,0.0,7.566468
min,14600.0,0.000000
25%,14600.0,56.000000
50%,14600.0,60.000000
75%,14600.0,64.000000
max,14600.0,74.000000


- 'days_employed' will be filled with 'median' value of its own column due to asymmetrical distribution
- 'days_employed' sera preenchido por valor de mediano devido sua distribuicao assimetrica

In [42]:
# filling null values with median value
# preenchendo valores nulos com mediana

cs['days_employed'] = cs['days_employed'].fillna(cs['days_employed'].median())

In [43]:
# finding unique values in cs'[age'] / achando valores unicos em cs['age']

cs['age'].sort_values().unique()

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75])

In [44]:
# contando quantos valores 0 para coluna 'age' / counting how many 0 values are for 'age' column

zero_age = 0

for row in cs['age']:
    
    if row == 0:
        zero_age +=1

zero_age
    

101

In [45]:
# substituindo valores 0 in 'age' por 18 / replace 0 values in 'age' for 18

cs['age'] = cs['age'].replace(0, 18)

In [46]:
# checando se valores for substituidos / checking if values were replaced

cs['age'].sort_values().unique()

array([18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75])

- 'total_income' will be filled with 'mean' value of its own column due to symmetrical distribution
- 'total_income' sera preenchido pelo valor medio devido sua distribuicao simetrica

In [48]:
# filling null valiues with mean value
# preenchendo valores nulos com media

cs['total_income'] = cs['total_income'].fillna(cs['total_income'].mean())

cs.isnull().sum()

children            0
days_employed       0
age                 0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [49]:
# defining very_low_income, low_income, medium_income, high_income and testing
# categorizando grupos em low_income, medium_income, high_income, very_high_income

def income_group(total_income):
    if total_income <= 20000:
        return 'low income'    
    if total_income <= 50000:
        return 'medium income'
    if total_income <= 100000:
        return 'high income'
    if total_income >= 150000:
        return 'very high income'

print(income_group(10000))
print(income_group(30000))
print(income_group(70000))
print(income_group(170000))

low income
medium income
high income
very high income


In [50]:
# adding column and aplying function
# adicionando coluna e aplicando funcao

cs['income_group'] = cs['total_income'].apply(income_group)

cs.head(10)    


,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,property,medium income
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car,low income
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,property,medium income
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,education,medium income
4,0,14600.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,medium income
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,property,medium income
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,property,medium income
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,medium income
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,wedding,low income
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,property,medium income


In [51]:
# filtrando por income_group e income_type para achar sua relacao / filtering by 'income_group and 'income_type to find their interaction

total_type_filtered = cs[['income_group', 'income_type']]

total_type_filtered.value_counts()

income_group      income_type                
medium income     employee                       6677
low income        employee                       3908
medium income     business                       3399
                  retiree                        1888
low income        retiree                        1851
                  business                       1127
medium income     civil servant                   870
high income       business                        503
                  employee                        503
low income        civil servant                   480
high income       retiree                         111
                  civil servant                   103
very high income  business                         18
                  employee                         10
low income        unemployed                        1
medium income     entrepreneur                      1
low income        paternity / maternity leave       1
medium income     unemployed        

In [52]:
cs.head()

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,property,medium income
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car,low income
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,property,medium income
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,education,medium income
4,0,14600.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,medium income


##  Is there any relation between having children and pay loan on time ?
 


In [54]:
# creating pivot table to calculate percentage of families in debt
# criando tabela pivot para calcular porcentagem de familias devedoras

pivot_table_children_debt = cs.pivot_table(index='children', values='debt', aggfunc='sum')

pivot_table_children_debt['total_families'] = cs['children'].value_counts()

pivot_table_children_debt['conversion'] = pivot_table_children_debt['debt'] / pivot_table_children_debt['total_families']

pivot_table_children_debt.loc[6, 'conversion'] = (pivot_table_children_debt['conversion'].sum() / 6)

pivot_table_children_debt.rename(index = { 6 : 'Average'}, inplace = True )

pivot_table_children_debt['Percentage'] = pivot_table_children_debt['conversion'].apply(lambda x: '{:.2%}'.format(x))

pivot_table_children_debt



,debt,total_families,conversion,Percentage
children,,,,
0,1063.0,14149.0,0.075129,7.51%
1,445.0,4865.0,0.091470,9.15%
2,202.0,2131.0,0.094791,9.48%
3,27.0,330.0,0.081818,8.18%
4,4.0,41.0,0.097561,9.76%
5,0.0,9.0,0.000000,0.00%
Average,NaN,NaN,0.073462,7.35%




Analisando os resultados da amostra, percentualmente.

Parece que não ter filhos diminui a chance de inadimplência comparado com famílias que tem apenas um filho em + 1.5%. 
O percentual é o menor entre os grupos dentro da amostra.

A chance de inadimplecia aumenta quando a família tem dois filhos em + 0.33% comparado com famílias de apenas um filho.

Curiosamente a inadimplência cai em -1.3% na amostra, em famílias que tem 3 filhos. Uma hipótese seria que famílias que decidem ter o 3o filho teriam mais estrutura.

A inadinplencia volta a subir para familias com 4 filhos para o seu patamar mais alto + 0.28% mais alto que familias com apenas dois filhos.

Já as famílias com 5 filhos, na nossa amostra, nenhuma das 9 estão em dívida.

Acredito que dentro da amostra, apenas 9 famílias, não tenha uma quantidade suficiente para detectar exatamente quais os níveis de inadimplência dentro desse grupo.

É possivel dizer que existe uma relação entre não pagar o empréstimo e ter filhos, a diferença entre não ter filhos e ter 4 filhos aumenta a chance de inadimplência em 2.25%.


## Is there any relation between total_income level and paying loan on time ?**

In [57]:
# finding what is the max value in 'total_income' / descobrindo valor maximo em 'total_income'

cs['total_income'].max()

362496.645

In [58]:
# finding minimum value in 'total_income' / descobrindo valor minimo em 'total_income'

cs['total_income'].min()

3306.762

In [59]:
# counting income_group / contando valores em 'income_group'

cs['income_group'].value_counts()

medium income       12836
low income           7369
high income          1221
very high income       28
Name: income_group, dtype: int64

In [60]:
cs['total_income'].describe()

count     21525.000000
mean      26787.568355
std       15621.268427
min        3306.762000
25%       17247.708000
50%       25024.051000
75%       31286.979000
max      362496.645000
Name: total_income, dtype: float64

In [61]:
# creating a pivot table to show percentage of 'income_groups' that have debt
# criando uma tabela pivor para calculares porcentagem de 'income_group' que tem divida

pivot_table_income_debt = cs.pivot_table(index='income_group', values='debt', aggfunc='sum')

pivot_table_income_debt['total_income_group'] = cs['income_group'].value_counts()

pivot_table_income_debt['conversion'] = pivot_table_income_debt['debt'] / pivot_table_income_debt['total_income_group']

pivot_table_income_debt.loc[4, 'conversion'] = (pivot_table_income_debt['conversion'].sum() / 4)

pivot_table_income_debt.rename(index = { 4 : 'Average'}, inplace = True )

pivot_table_income_debt['Percentage'] = pivot_table_income_debt['conversion'].apply(lambda x: '{:.2%}'.format(x))

pivot_table_income_debt

,debt,total_income_group,conversion,Percentage
income_group,,,,
high income,86.0,1221.0,0.070434,7.04%
low income,608.0,7369.0,0.082508,8.25%
medium income,1041.0,12836.0,0.081100,8.11%
very high income,2.0,28.0,0.071429,7.14%
Average,NaN,NaN,0.076368,7.64%



Analisando o resultados da amostra percentualmente.

O grupo com menor renda tem o mais alto percentual de inadimplência, ficando 0.14% acima do grupo de renda média.

O maior salto para a não inadimplecia, passa do grupo de renda média para o grupo com renda alta, a diferença abaixa em 1.07%.

Sobre o grupo de renda altíssima, percentualmente comparado com o grupo de renda alta, a inadimplência aumenta em 0.10%. Uma hipotese seria que os investimentos feitos por famílias
de alta renda, talvez contenham mais riscos do que as famílias de renda alta.

É possível dizer que existe relação direta entre inadimplencia e renda. Sendo que a diferença entre o grupo de renda baixa com o de renda alta chega a 1.21%.







## Is there any relation between civil status and paying loan on time ?

In [64]:
# creating a pivot table to show percentage of family status that have debt
# criando uma tabela pivot para calcular porcentagem de 'family_status' que tem divida

pivot_table_family_status_debt = cs.pivot_table(index='family_status', values='debt', aggfunc='sum')

pivot_table_family_status_debt['total_family_status'] = cs['family_status'].value_counts()

pivot_table_family_status_debt['conversion'] = pivot_table_family_status_debt['debt'] / pivot_table_family_status_debt['total_family_status']

pivot_table_family_status_debt.loc[5, 'conversion'] = (pivot_table_family_status_debt['conversion'].sum() / 5)

pivot_table_family_status_debt.rename(index = { 5 : 'Average'}, inplace = True )

pivot_table_family_status_debt['Percentage'] = pivot_table_family_status_debt['conversion'].apply(lambda x: '{:.2%}'.format(x))

pivot_table_family_status_debt

,debt,total_family_status,conversion,Percentage
family_status,,,,
civil partnership,388.0,4177.0,0.092890,9.29%
divorced,85.0,1195.0,0.071130,7.11%
married,931.0,12380.0,0.075202,7.52%
unmarried,274.0,2813.0,0.097405,9.74%
widow / widower,63.0,960.0,0.065625,6.56%
Average,NaN,NaN,0.080450,8.05%



Analisando os dados da amostra, percentualmente.

As viúvas/viúvos é o grupo com menor inadimplência dentro da amostra. Apenas, 6.56%

Os divorciados vem a seguir com o nível 0.55% mais alto que o menor do grupo.

Na sequência temos o grupo dos casados, apenas 0.41% acima do grupo anterior, o que representa a menor diferença entre os grupos da amostra.

Os em parceria cívil junto com os não casados apresentam a maior taxa de inadimplência, acima dos 9%

Podemos concluir que existe uma relação entre ser inadimplente e ser casado ou não. A chance de inadimplência aumenta quando não casado. Sendo que a difenrença percentual entra o grupo mais inadimplente e o menos chega a 3.18%.


## How different reasons to take a loan affect its payment in time ?


In [67]:
# creating a pivot table to show percentage of 'purposes' that have debt
# criando tabela pivot para calcular porcentagem de 'purposes' que tem divida

pivot_table_purpose_debt = cs.pivot_table(index='purpose', values='debt', aggfunc='sum')

pivot_table_purpose_debt['total_purpose'] = cs['purpose'].value_counts()

pivot_table_purpose_debt['conversion'] = pivot_table_purpose_debt['debt'] / pivot_table_purpose_debt['total_purpose']

pivot_table_purpose_debt.loc[5, 'conversion'] = (pivot_table_purpose_debt['conversion'].sum() / 5)

pivot_table_purpose_debt.rename(index = { 5 : 'Average'}, inplace = True )

pivot_table_purpose_debt['Percentage'] = pivot_table_purpose_debt['conversion'].apply(lambda x: '{:.2%}'.format(x))

pivot_table_purpose_debt

,debt,total_purpose,conversion,Percentage
purpose,,,,
building,225.0,3140.0,0.071656,7.17%
car,403.0,4315.0,0.093395,9.34%
education,370.0,4022.0,0.091994,9.20%
property,557.0,7700.0,0.072338,7.23%
wedding,186.0,2348.0,0.079216,7.92%
Average,NaN,NaN,0.081720,8.17%


Analisando os dados da amostra, percentualmente.

O grupo com menor inadimplência são os que pegaram o emprestímo para construção ou reforma da propriedade. E logo na sequência sendo apenas +0.06% mais inadimplente, os que pegaram o empréstimo para comprar uma propriedade.

Os grupos com maior inadimplência são os que pegaram seus emprestimos para investir na educação, ou comprar um carro. O salto dos que pegaram o emprestimo para casar para o próximo mais inadimplente é de +1.28%

É possível dizer que a relação entre os motivos e entre pagar ou não pode estar associada ao momento na vida de quem pega o emprestimo. 

Geralmente ainda no começo da vida adulta é quando compramos um carro e temos que pagar pelos nossos estudos, portanto, ainda há muita fragilidade financeira dentro desse grupo.

Vemos então que o grupo dos que pegaram o emprestimo para casar, construir ou comprar uma propriedade, por ja estarem, provavelmente mais maduros e mais avançados em suas carreiras profissionais, tendem a cumprir o pagamento de suas dívidas.





## How different education levels affect loan payment?


In [70]:
# creating a pivot table to show percentage of 'education' that have debt
# criando tabela pivot para calcular porcentagem de 'education' que tem divida


pivot_table_education_debt = cs.pivot_table(index='education', values='debt', aggfunc='sum')

pivot_table_education_debt['total_education'] = cs['education'].value_counts()

pivot_table_education_debt['conversion'] = pivot_table_education_debt['debt'] / pivot_table_education_debt['total_education']

pivot_table_education_debt.loc[5, 'conversion'] = (pivot_table_education_debt['conversion'].sum() / 5)

pivot_table_education_debt.rename(index = { 5 : 'Average'}, inplace = True )

pivot_table_education_debt['Percentage'] = pivot_table_education_debt['conversion'].apply(lambda x: '{:.2%}'.format(x))



pivot_table_education_debt

,debt,total_education,conversion,Percentage
education,,,,
bachelor's degree,278.0,5260.0,0.052852,5.29%
graduate degree,0.0,6.0,0.000000,0.00%
primary education,31.0,282.0,0.109929,10.99%
secondary education,1364.0,15233.0,0.089542,8.95%
some college,68.0,744.0,0.091398,9.14%
Average,NaN,NaN,0.068744,6.87%



Analisando os dados da amostra, percentualmente.

Podemos ver que os níveis mais altos de inadimplência estão no grupo com apenas educação primária.

Após, encontramos o grupo que começou a universidade, mas não terminou. A diferença varia em 1.85% menos inadimplente que o pior grupo.

O grupo com educação secundária, se encontra muito próximo do grupo com universidade inacabada, variação de apenas 0.19%

Drasticamente o numero cai para o grupo que tem bacharelado, a diferença entre ele e o próximo mais inadimplente é de 3.66%.

E o grupo graduado, não possui dívida, embora, acredite que a quantidade da amostra para esse grupo tenha sido muito pequena para que se possa ter um número mais acurado.

Podemos concluir que existe uma relação direta entre esses grupos, e se irão ou não conseguir pagar seus emprestimos.
O grupo com maior escolariadade apresenta melhor crédtio.



# Conclusion



Concluimos que, após avaliar todas as hipoteses dentro dos dados obtidos da amostra.

O grupo com menor inadimplência são os graduados com 5.29%.

O grupo com maior inadimplência são os que têm apenas educação primária com 10.99%

O que nos indica que o nivel educacional é um fator preponderante na hora de analisar o 'score' de quem pede o empréstimo.

A comparacao media entre os grupos coloca, a analise por motivos como a que tem a maior media.











